In [34]:
import pandas as pd
import numpy as np

In [37]:
# Load lake data
lakedata = pd.read_csv('lake_data.csv')
lakedata_df = pd.DataFrame(lakedata)
display(lakedata_df)

,T_C,depth_cm
0,-2.00,0
1,1.78,1
2,5.56,5
3,9.33,10
4,13.11,20
5,16.89,50
6,20.67,100
7,24.44,200
8,28.22,500
9,32.00,1000


In [38]:
def clumped_sensor(T_C, model='I-CDES'):
    if model == 'I-CDES':
        Cap47 = 0.0004 * ((10**6) / (T_C**2)) + 0.154  # not including +/- uncertainties, can add later
    elif model == 'CDES90': 
        Cap47 = 1  # will add other functions
    else:
        raise ValueError("Model not recognized")
    return Cap47 # returns model eq back to caller (clumped_sensor)

In [39]:
# Calculate mean & stdev for depth and temperature columns
def generate_statistics(lakedata_df): 
    stats = { 'depth_cm_mean': round(lakedata_df['depth_cm'].mean(), 2), 
             'depth_cm_std': round(lakedata_df['depth_cm'].std(), 2),
             'T_C_mean': round(lakedata_df['T_C'].mean(), 2),
             'T_C_std': round(lakedata_df['T_C'].std(), 2)
            }
    return stats 

statistics = generate_statistics(lakedata_df) 
statistics_df = pd.DataFrame([statistics])
display(statistics_df)

# Create weight_dict with the statistics (parameters for the normal_dist weighting function)
weight_dict = {
    'depth_cm_mean': statistics['depth_cm_mean'],
    'depth_cm_std': statistics['depth_cm_std'],
    'T_C_mean': statistics['T_C_mean'],
    'T_C_std': statistics['T_C_std']}

,depth_cm_mean,depth_cm_std,T_C_mean,T_C_std
0,188.6,324.44,15.0,11.44


In [46]:
depths = np.array(lakedata_df['depth_cm'])

def generate_weights(depths, weight_type, weight_dict):
    """
    Generate a depth-weighting function.

    Args:
        depths (np.ndarray or pd.Series): Array or DataFrame column of depth values.
        weight_type (str): Equation type for computing the weighting with depth.
        weight_dict (dict): Dictionary of input parameter values for the weighting equation.

    Returns:
        np.ndarray: Array of weights aligned with the depth values.
    """
    if weight_type == "uniform":
        weights = np.repeat(1 / len(depths), len(depths))
       
    elif weight_type == "surface":
        weights = np.array([1] + [0] * (len(depths) - 1))
       
    elif weight_type == "normal_dist":
        depth_cm_mean = weight_dict["depth_cm_mean"]
        depth_cm_std = weight_dict["depth_cm_std"]
        weights = (1 / (depth_cm_std * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((depths - depth_cm_mean) / depth_cm_std) ** 2)
        # Normalize the weights so they sum to 1
        weights /= weights.sum()
    else:
        raise ValueError("Weight type not recognized")
    # Add more conditions for other weight types if needed

    return weights

In [48]:
# Calculate Cap47 at each depth and add it to lakedata_df
lakedata_df['Cap47'] = lakedata_df['T_C'].apply(clumped_sensor)
    
#Calculate Cap47 weights
lakedata_df['weights'] = generate_weights(depths, "normal_dist", {"depth_cm_mean": weight_dict['depth_cm_mean'], "depth_cm_std": weight_dict['depth_cm_std']})

# Calculate weighted mean of Cap47 for water column
lakedata_df['Cap47_wtd_mean'] = ((lakedata_df['Cap47'] * lakedata_df['weights']).sum())/lakedata_df.shape[1]  

display(lakedata_df)

,T_C,depth_cm,Cap47,weights,Cap47_wtd_mean
0,-2.00,0,100.154000,0.107914,5.434761
1,1.78,1,126.400686,0.108107,5.434761
2,5.56,5,13.093289,0.108873,5.434761
3,9.33,10,4.749118,0.109813,5.434761
4,13.11,20,2.481312,0.111639,5.434761
5,16.89,50,1.556170,0.116635,5.434761
6,20.67,100,1.090222,0.123102,5.434761
7,24.44,200,0.823665,0.127700,5.434761
8,28.22,500,0.656280,0.080615,5.434761
9,32.00,1000,0.544625,0.005601,5.434761
